In [23]:
import  pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from transformers import BertTokenizer, BertForMaskedLM
import torch
import string

In [24]:
def load_model(model_name):
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model

In [25]:
def get_prediction_eos(input_text):
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res

In [26]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    if tokenizer.mask_token == text_sentence.split()[-1]:
      text_sentence += ' .'
      input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
      mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [28]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
       token = ''.join(tokenizer.decode(w).split())
       if token not in ignore_tokens:
          tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [35]:
def get_all_predictions(text_sentence, top_clean):
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = bert_model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
    return {'bert': bert}    

In [36]:
top_k=5
model_name='BERT'
bert_tokenizer, bert_model  = load_model(model_name)
input_text="Fuck me"
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
    answer.append(i)
answer_as_string = "    ".join(answer)
print("Predicted List is Here",answer_as_string )   

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['now', 'tonight', 'again', 'hard', 'then']
Predicted List is Here now    tonight    again    hard    then
